# Demonstration Creating BenchmarkSystems and Related Functions

## Import Required Libraries

In this section, we import the necessary libraries and modules required for the demonstration.

In [1]:
import json
import copy

from openfe_benchmarks.scripts.calculation_setup import RBFEBenchmarkSystem
from openfe import SmallMoleculeComponent, SolventComponent, ProteinComponent
from openff.toolkit.utils.toolkit_registry import ToolkitRegistry
from openff.toolkit import RDKitToolkitWrapper, NAGLToolkitWrapper

from openfe_benchmarks.data import get_benchmark_data_system
from openfe_benchmarks.scripts.utils import (
    generate_relative_network_from_names, update_partial_charges, assign_partial_charges, process_sdf
)

## Demonstrate RBFEBenchmarkSystem

Create an instance of `RBFEBenchmarkSystem` and demonstrate its initialization, usage, and customization.

In [2]:
benchmark_data = get_benchmark_data_system("industry_benchmark_systems.mcs_docking_set", "hne")
solvent = SolventComponent(positive_ion='Na', negative_ion='Cl', neutralize=True)
forcefield = 'openff-2.3.0'
partial_charge_scheme = "nagl_openff-gnn-am1bcc-1.0.0.pt"

2026-01-29 18:23:51 | INFO     | Loaded system 'hne' from benchmark set 'industry_benchmark_systems.mcs_docking_set' with 5 ligand file(s), and 5 cofactor file(s). Found protein file: False.


In [3]:
rbfe_system = RBFEBenchmarkSystem(
    benchmark_data, partial_charge_scheme=partial_charge_scheme, solvent=solvent, forcefield=forcefield
)
rbfe_system.generate_alchemical_network()
rbfe_system.network

<AlchemicalNetwork-40192345c11061ec55f0225083fd3de8>

In [4]:
# Now save to a file
rbfe_system.export_alchemical_network()

## Generate Relative Network

Use `generate_relative_network_from_names` to create a custom ligand network for an rbfe calculation.

In [5]:
# Generate new network
rbfe_copy1 = copy.deepcopy(rbfe_system)
ligand_names = list(rbfe_copy1.ligand_dict.keys())
connections = [(ligand_names[0], ligand_names[1])]

rbfe_copy1.initial_network = generate_relative_network_from_names(
    rbfe_copy1.ligand_dict, connections
)
rbfe_copy1.generate_alchemical_network()
rbfe_copy1.network

6 14


/Users/jenniferclark/bin/gufe/gufe/components/explicitmoleculecomponent.py:71: UserWarning: RDKit does not preserve Mol properties when pickled by default, which may drop e.g. atom charges; consider setting `Chem.SetDefaultPickleProperties(Chem.PropertyPickleOptions.AllProps)`
  warnings.warn(


<AlchemicalNetwork-30d7721d497511323ba654010b078c5f>

## Update Partial Charges

You can also specify custom partial charges for your ligands and cofactors.

In [6]:
# Update partial charges
rbfe_copy2 = copy.deepcopy(rbfe_system)
rbfe_copy2.ligand_dict = {lig.name: lig for lig in update_partial_charges(
    rbfe_copy1.ligand_dict.values(), 
    partial_charge_method='openff-gnn-am1bcc-0.1.0-rc.3.pt', 
    off_wrapper=ToolkitRegistry([RDKitToolkitWrapper(), NAGLToolkitWrapper()]),
)}
rbfe_copy1.generate_alchemical_network()
rbfe_copy1.network

[18:23:57] WARNING: Charges were rearranged

[18:23:59] WARNING: Proton(s) added/removed

[18:24:01] WARNING: Proton(s) added/removed

[18:24:02] WARNING: Proton(s) added/removed



<AlchemicalNetwork-30d7721d497511323ba654010b078c5f>